# Synthetix V3


In [13]:
import os
import time
from synthetix import Synthetix
from synthetix.utils import wei_to_ether, ether_to_wei, format_wei, format_ether
from synthetix.utils.multicall import call_erc7412, multicall_erc7412
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
snx = Synthetix(
    provider_rpc=os.getenv("LOCAL_RPC"),
    network_id=11155111,
    is_fork=True,
    request_kwargs={"timeout": 120},
    # private_key=os.getenv("PRIVATE_KEY"),
    # cannon_config={
    #     "package": "synthetix-omnibus",
    #     "version": "latest",
    #     "preset": "main",
    # },
    # carina deployment
    # cannon_config={"ipfs_hash": "QmWNC8PGBTwXc2DgjVXDpN1A9TxNR9UusKyyoyNmm2j982"},
    # local deployment
    ipfs_gateway="http://127.0.0.1:8080/ipfs/",
    cannon_config={"ipfs_hash": "QmdsxSMNuz2EvRY98yWsSbiFd1me68C59h5Ftfv5k13ULG"},
)

2024-07-25 15:46:29,107 - INFO - Using RPC signer: 0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
2024-07-25 15:46:29,217 - INFO - Loading cannon contracts at ipfs hash ipfs://QmdsxSMNuz2EvRY98yWsSbiFd1me68C59h5Ftfv5k13ULG


In [ ]:
snx.perps.markets_by_id

In [ ]:
snx.perps.market_meta[3]["config"]["pyth_publish_time_min"]

In [ ]:
snx.perps.market_meta[3]["config"]["pyth_publish_time_max"]

In [ ]:
snx.contracts.keys()

In [ ]:
from synthetix.contracts import fetch_raw_deploy_from_ipfs
import json

raw_data = fetch_raw_deploy_from_ipfs(
    snx, "QmYNLUWFfhvotaSNgKg57uArqriZsPWGaspNzUAuDd5AfC"
)
raw_json = json.dumps(raw_data)
raw_json

In [3]:
def find_contract_with_function(contracts, function_signature):
    def search_recursive(item, path=[]):
        if isinstance(item, dict):
            if "contract" in item and "abi" in item and "address" in item:
                # This is a contract
                contract = item["contract"]
                try:
                    function = contract.get_function_by_signature(function_signature)
                    if function:
                        return ".".join(path), contract
                except Exception as e:
                    # If get_function_by_signature raises an exception, we'll just continue
                    pass
            else:
                # This might be a package or a nested structure
                for key, value in item.items():
                    result = search_recursive(value, path + [key])
                    if result:
                        return result
        return None

    result = search_recursive(contracts)
    return result if result else (None, None)

In [4]:
find_contract_with_function(snx.contracts, "0xd684bd92")

(None, None)

In [ ]:
snx.perps.markets_by_id

In [ ]:
pyth = snx.contracts["Pyth"]["contract"]

In [ ]:
snx.perps.market_proxy.functions.getMarketConfiguration().call()

In [ ]:
snx.perps.markets_by_id

In [ ]:
market_configs = call_erc7412(
    snx, snx.perps.market_proxy, "getMarketConfigurationById", (3,)
)

In [ ]:
market_configs

In [ ]:
snx.perps.market_proxy.get_function_by_name("getMarketConfigurationById").abi

In [ ]:
snx.perps.markets_by_name

In [ ]:
snx.contracts["system"]["USDProxy"]["contract"]

In [ ]:
snx.contracts.keys()

In [ ]:
snx.core.core_proxy.functions.getCollateralConfigurations(False).call()

In [ ]:
snx.perps.market_proxy.functions.minimumCredit(3).call()

In [ ]:
snx.perps.create_account(submit=True)

In [ ]:
snx.perps.account_ids

In [ ]:
snx.perps.modify_collateral(
    90000,
    collateral_address=snx.contracts["system"]["USDProxy"]["contract"].address,
    market_id=1,
    account_id=snx.perps.account_ids[-1],
    submit=True,
)

In [ ]:
oracle_price = snx.perps.markets_by_id[1]["oracle_price"]
position_size = ether_to_wei(1 / oracle_price)
limit_price = ether_to_wei(oracle_price * 1.01)
perps_account_id = snx.perps.account_ids[0]

# commit_tx = snx.perps.commit_order(
#     position_size,
#     market_id=1,
#     limit_price=limit_price,
#     account_id=perps_account_id,
#     submit=True,
# )
# commit_receipt = snx.wait(commit_tx)

In [ ]:
tx_params = snx._get_tx_params(to=snx.perps.market_proxy.address)
tx_params["data"] = snx.perps.market_proxy.encodeABI(
    fn_name="commitOrder",
    args=[
        perps_account_id,
        1,
        position_size,
        limit_price,
        0,
        [],
    ],
)

# tx_hash = snx.execute_transaction(tx_params)

In [ ]:
tx_params

In [ ]:
tx_hash = snx.execute_transaction(tx_params)

In [ ]:
snx.web3.eth.estimate_gas(tx_params)

In [ ]:
snx.perps.get_open_position(market_id=1)

In [ ]:
snx.perps.get_quote(1, market_id=1)

In [ ]:
tx_params = snx.perps.commit_order(1, market_id=1)

### Setup


In [5]:
# get some contracts
weth = snx.contracts["weth_mock_collateral"]["MintableToken"]["contract"]
snx.contracts["WETH"] = snx.contracts["weth_mock_collateral"]["MintableToken"]
susd = snx.contracts["system"]["USDProxy"]["contract"]
snx_contract = snx.contracts["SNX"]["contract"]

In [6]:
weth.address

'0x5f75Ea59fC0CF335E09775eB676cd53c1C49Cbf0'

In [7]:
def status():
    """A function printing some useful information about the connected account"""
    eth_balance = snx.get_eth_balance()
    weth_balance = weth.functions.balanceOf(snx.address).call() / 1e18
    susd_balance = susd.functions.balanceOf(snx.address).call() / 1e18
    snx_balance = snx_contract.functions.balanceOf(snx.address).call() / 1e18

    print(
        f"""
    ETH: {eth_balance['eth']}
    WETH ({weth.address}): {weth_balance}
    SNX ({snx_contract.address}): {snx_balance}
    
    sUSD: {susd_balance}
    """
    )


status()


    ETH: 10000.0
    WETH (0x5f75Ea59fC0CF335E09775eB676cd53c1C49Cbf0): 0.0
    SNX (0x079E7E1C0cBFee4B2D2798A9d426009D08b1F9cb): 0.0
    
    sUSD: 0.0
    


In [ ]:
snx.perps

In [ ]:
snx.perps.market_proxy.all_functions()

In [ ]:
snx.perps.market_proxy.functions.name(0).call()

In [ ]:
snx.perps.market_proxy.functions.getMarketConfiguration().call()

In [ ]:
snx.perps.market_proxy.functions.getMarketDigest(3).call()

In [ ]:
snx.perps.get_margin_info(market_id=3)

In [10]:
pyth_feed_id = snx.perps.markets_by_id[3]["feed_id"]
pyth_data = snx.pyth.get_price_from_ids([pyth_feed_id])

2024-07-25 15:31:57,436 - INFO - Fetching Pyth data for 1 markets


In [12]:
pyth_data["meta"][pyth_feed_id]["price"]

3174.97447266

In [16]:
pyth = snx.contracts["Pyth"]["contract"]